In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import isnan, when, count, col
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import sklearn 
from sklearn.metrics import classification_report, confusion_matrix

In [0]:
spark = SparkSession.builder.appName('CCDefault').getOrCreate()

In [0]:
#Part 1 - Data Preprocessing
df = spark.read.csv('/FileStore/tables/UCI_Credit_Card.csv',inferSchema=True,header=True)

df.printSchema()

root
-- ID: integer (nullable = true)
-- LIMIT_BAL: double (nullable = true)
-- SEX: integer (nullable = true)
-- EDUCATION: integer (nullable = true)
-- MARRIAGE: integer (nullable = true)
-- AGE: integer (nullable = true)
-- PAY_0: integer (nullable = true)
-- PAY_2: integer (nullable = true)
-- PAY_3: integer (nullable = true)
-- PAY_4: integer (nullable = true)
-- PAY_5: integer (nullable = true)
-- PAY_6: integer (nullable = true)
-- BILL_AMT1: double (nullable = true)
-- BILL_AMT2: double (nullable = true)
-- BILL_AMT3: double (nullable = true)
-- BILL_AMT4: double (nullable = true)
-- BILL_AMT5: double (nullable = true)
-- BILL_AMT6: double (nullable = true)
-- PAY_AMT1: double (nullable = true)
-- PAY_AMT2: double (nullable = true)
-- PAY_AMT3: double (nullable = true)
-- PAY_AMT4: double (nullable = true)
-- PAY_AMT5: double (nullable = true)
-- PAY_AMT6: double (nullable = true)
-- default_payment: integer (nullable = true)

In [0]:
#1.1 Cleaning Null Values
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show() #no null values

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+---------------+
 ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|default_payment|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+---------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+---------------+

In [0]:
#1.2 Creating Features Column

assembler = VectorAssembler(inputCols=df.columns[1:-1],
                           outputCol='features') 
#so we're grabbing columns 1 to 2nd last one which are our features. Last column is our label. Column 0 is ID which is not relevant

output = assembler.transform(df) #applying our vector assembler to all our data

output.select('features').head(1) #notice features is a DenseVector containing all the features we combined

Out[6]: [Row(features=DenseVector([20000.0, 2.0, 2.0, 1.0, 24.0, 2.0, 2.0, -1.0, -1.0, -2.0, -2.0, 3913.0, 3102.0, 689.0, 0.0, 0.0, 0.0, 0.0, 689.0, 0.0, 0.0, 0.0, 0.0]))]

In [0]:
#1.2 Creating Features and Label Dataframe
output.groupBy('default_payment').count().show() #so we got 2 categories for the label

+---------------+-----+
default_payment|count|
+---------------+-----+
 1| 6636|
 0|23364|
+---------------+-----+

In [0]:
df_final = output.select('features', 'default_payment') #so the data we'll be using will be the features and the label

df_final.show(3)

+--------------------+---------------+
 features|default_payment|
+--------------------+---------------+
[20000.0,2.0,2.0,...| 1|
[120000.0,2.0,2.0...| 1|
[90000.0,2.0,2.0,...| 0|
+--------------------+---------------+
only showing top 3 rows

In [0]:
#1.5 Train Test Split
train, test = df_final.randomSplit([0.75,0.25]) #train will have 80%, test will have 20% of data

train.describe().show()

+-------+-------------------+
summary| default_payment|
+-------+-------------------+
 count| 22386|
 mean|0.22129902617707495|
 stddev|0.41513066070799065|
 min| 0|
 max| 1|
+-------+-------------------+

In [0]:
test.describe().show()

+-------+-------------------+
summary| default_payment|
+-------+-------------------+
 count| 7614|
 mean|0.22090885211452588|
 stddev| 0.4148864161533842|
 min| 0|
 max| 1|
+-------+-------------------+

In [0]:
#Part 2 - Creating our Model
#2.1 Logistc Regression Model
logreg = LogisticRegression(featuresCol='features', labelCol='default_payment', predictionCol='prediction', maxIter=200)

logreg_model = logreg.fit(train) #fitting our train data

In [0]:
pred_logreg = logreg_model.evaluate(test)

y_true_logreg = pred_logreg.predictions.select(['default_payment']).collect()
y_pred_logreg = pred_logreg.predictions.select(['prediction']).collect()

print(classification_report(y_true_logreg, y_pred_logreg))

precision recall f1-score support

 0 0.82 0.97 0.89 5932
 1 0.70 0.25 0.37 1682

 accuracy 0.81 7614
 macro avg 0.76 0.61 0.63 7614
weighted avg 0.79 0.81 0.77 7614

In [0]:
print(confusion_matrix(y_true_logreg, y_pred_logreg))

[[5754 178]
 [1266 416]]

In [0]:
#2.2 Random Forest Model
rf = RandomForestClassifier(labelCol="default_payment", featuresCol="features", numTrees=10)

rf_model = rf.fit(train)

In [0]:
pred_rf = rf_model.evaluate(test)

y_true_rf = pred_rf.predictions.select(['default_payment']).collect()
y_pred_rf = pred_rf.predictions.select(['prediction']).collect()

print(classification_report(y_true_rf, y_pred_rf))

precision recall f1-score support

 0 0.84 0.95 0.89 5932
 1 0.68 0.34 0.45 1682

 accuracy 0.82 7614
 macro avg 0.76 0.65 0.67 7614
weighted avg 0.80 0.82 0.79 7614

In [0]:
print(confusion_matrix(y_true_rf, y_pred_rf))

[[5665 267]
 [1111 571]]

In [0]:
rf_model.featureImportances #looks like index 5 (Column PAY_0) was the largest determining factor

Out[27]: SparseVector(23, {0: 0.0327, 1: 0.0002, 2: 0.0018, 4: 0.0044, 5: 0.4052, 6: 0.1294, 7: 0.1841, 8: 0.0464, 9: 0.0219, 10: 0.0792, 11: 0.0072, 12: 0.0041, 13: 0.0054, 14: 0.0079, 15: 0.0035, 16: 0.0012, 17: 0.0332, 18: 0.0104, 19: 0.0062, 20: 0.0101, 21: 0.0027, 22: 0.0026})

In [0]:
#2.3 Decision Tree Model
dt = DecisionTreeClassifier(labelCol="default_payment", featuresCol="features")

dt_model = dt.fit(train)

In [0]:
pred_dt = dt_model.transform(test)

evaluator = MulticlassClassificationEvaluator(
    labelCol="default_payment", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(pred_dt)
print("Test Error = %g " % (1.0 - accuracy))

Test Error = 0.179275

In [0]:
dt_model.featureImportances #looks like index 5 (Column PAY_0) was the largest determining factor once again

Out[36]: SparseVector(23, {0: 0.0089, 2: 0.0072, 4: 0.003, 5: 0.7789, 6: 0.1543, 7: 0.0128, 10: 0.0157, 11: 0.0046, 13: 0.0037, 16: 0.002, 18: 0.0022, 19: 0.0017, 20: 0.0051})